# Starting a new EvoMol optimization where a previous one stopped

In this tutorial, we show how it is possible to start a new optimization procedure using as initial population the molecules that were in the population at the end of a previous EvoMol run. We also cover the case where we build a cache in order to avoid another calculation of DFT-dependent properties.

## General case

### Base optimization

Let's execute our base optimization instance. Here we maximize the HOMO energy value for 10 optimization steps.

In [1]:
from evomol import run_model

model_path = "./data/homo_max_1s"

In [ ]:
run_model({
    "obj_function": "homo",
    "io_parameters":{
        "model_path": model_path
    },
    "optimization_parameters":{
        "max_steps": 10
    }
})

### New execution starting from previous experiment's data

Now we would want to know what would have happened if the previous experiment had been running for a longer time. We start a new experiment that runs for 20 additional steps, using as initial population the final population from previous experiment.

We use for this the *smiles_list_init_path* parameter, that can accept the path to a file that contain a list of SMILES, but also in our case the path to the population file (pop.csv) of a previous EvoMol experiment.

Note that here **the objective function values of the initial population must be computed again**, which is unfortunate since the objective function depends on costly DFT calculations. This particular issue is addressed in next cell

In [ ]:
from os.path import join

new_model_path = "./data/homo_max_2s"

run_model({
    "obj_function": "homo",
    "io_parameters":{
        "model_path": new_model_path,
        "smiles_list_init_path": join(model_path, "pop.csv")
    },
    "optimization_parameters":{
        "max_steps": 20
    }
})

## Using a cache to avoid another computation of the electronic properties

Now, we want to launch a final optimization starting from the population of the previous experiment. This time, we start by building a cache of all electronic properties values that are stored in the *pop.csv* file.

### Building the cache

We define the following function, that depends on a list of electronic properties to be extracted. Here we only need the HOMO ("homo") value. In other contexts, the other properties could be also retrieved ("lumo" for LUMO, "gap" for HOMO/LUMO gap and "homo-1" for HOMO-1).

In [7]:
import pandas as pd
import json 

def create_cache_OPT(model_path, output_cache_path, properties_list):
    
    # Loading pop.csv file
    df = pd.read_csv(join(model_path, "pop.csv"))
    
    cache_dict = {}
    
    # Iterating over all SMILES
    for i, smi in enumerate(df["smiles"]):
        curr_dict = {}
        
        # Iterating over all electronic properties to be cached
        for prop in properties_list:
            curr_dict[prop] = df[prop][i]
        
        # Saving cached values for current SMILES
        cache_dict[smi] = curr_dict
    
    # Writing the cache data to a JSON file
    with open(output_cache_path, "w") as f:
        json.dump(cache_dict, f)



In [8]:
cache_path = "./data/homo_max_2s_cache.json"

create_cache_OPT(new_model_path, cache_path, ["homo"])

### Running the final experiment using the cache

Now we run the final experiment performing 10 additional optimization steps. This time, we use the cache we built ("dft_cache_files" parameter) in order not to compute the properties of the initial population again.

In [9]:
final_model_path = "./data/homo_max_3s"

run_model({
    "obj_function": "homo",
    "io_parameters":{
        "model_path": new_model_path,
        "smiles_list_init_path": join(model_path, "pop.csv"),
        "dft_cache_files": [cache_path]
    },
    "optimization_parameters":{
        "max_steps": 10
    }
})

DFT MM obabel_mmff94
256 molecules in cache
SYMBOLS LIST : ['C', 'N', 'O', 'F', 'P', 'S', 'Cl', 'Br']
objective_calls
Computing scores at initialization...
Start pop algorithm
homo_mean : -7.05545
homo_med : -6.81019
homo_std : 1.19662
homo_min : -10.58115
homo_max : -4.92526
total_mean : -7.05545
total_med : -6.81019
total_std : 1.19662
total_min : -10.58115
total_max : -4.92526
new step
step : 0
best : C=CN
computing dft for CC=N
Starting OPT
Execution time OPT: 10s
There are 8 atoms and 37 MOs
computing dft for N=P
MM error (Evaluation error)
computing dft for C=NN
Starting OPT


1 molecule converted


Execution time OPT: 9s
There are 7 atoms and 35 MOs
computing dft for CN(C)F
Starting OPT


1 molecule converted


Execution time OPT: 22s
There are 10 atoms and 48 MOs
computing dft for C=NBr
MM error (Evaluation error)
computing dft for CC(N)Cl


...

1 molecule converted


computing dft for CC(N)C(N)(N)Br
Starting OPT
Execution time OPT: 420s


1 molecule converted


There are 17 atoms and 103 MOs
computing dft for NCCCCN
Starting OPT
Execution time OPT: 55s
There are 18 atoms and 78 MOs
homo_mean : -6.41614
homo_med : -6.13889
homo_std : 1.04769
homo_min : -10.58115
homo_max : -4.92200
total_mean : -6.41614
total_med : -6.13889
total_std : 1.04769
total_min : -10.58115
total_max : -4.92200
Stopping : stop condition reached


1 molecule converted
